In [ ]:
%matplotlib inline

In [ ]:
# import libraries
import netCDF4
import os, glob
import cv2
import rasterio
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from simpledbf import Dbf5
from datetime import datetime, timedelta
from sklearn.preprocessing import OneHotEncoder

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


In [ ]:
import timm

# SEED 고정 및 Params 입력

In [ ]:
# Random Seed Initialize
RANDOM_SEED = 21

def seed_everything(seed=RANDOM_SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
EPOCHS=1
BATCH_SIZE=128
LEARNING_RATE=0.001
device= 'cuda:0'

eps= 1e-6

# 데이터 준비

In [ ]:
tpw = glob.glob('/data2/yeonsik/rain/TPW/*/*/*.nc')
rr = glob.glob('/data2/yeonsik/rain/RR/*/*.nc')
gpm = glob.glob('./train/input_nasa_gpm/*/*.tif')
gpm_test = glob.glob('./test/input_nasa_gpm/*.tif')
gpm += gpm_test
tpw.sort()
rr.sort()
gpm.sort()

In [ ]:
rr_table = pd.DataFrame({
    'date' : list(map(lambda x: datetime(int(x[-15:-11]), int(x[-11:-9]), int(x[-9:-7]), int(x[-7:-5])), rr)),
    'rr' : rr
})

tpw_table = pd.DataFrame({
    'date' : list(map(lambda x: datetime(int(x[-15:-11]), int(x[-11:-9]), int(x[-9:-7]), int(x[-7:-5])), tpw)),
    'tpw' : tpw
})


gpm_h = []
gpm_time = []
for i in range(0,len(gpm),2):
    gpm_h.append([gpm[i], gpm[i+1]])
    gpm_time.append(datetime(int(gpm[i+1][-44:-40]), int(gpm[i+1][-40:-38]), int(gpm[i+1][-38:-36]), int(gpm[i+1][-26:-24]), int(gpm[i+1][-24:-22])) + timedelta(minutes=1) + timedelta(hours=9))

gpm_table = pd.DataFrame({
    'date' : gpm_time,
    'gpm' : gpm_h
})

tmp = pd.merge(rr_table, tpw_table, on='date')

In [ ]:
train_output = pd.read_csv('./train/train_output_seomjingang.csv')
test_output = pd.read_csv('./test/sample_output_seomjingang.csv')
del train_output['비고']
del test_output['비고']
train_output.columns = ['code', 'region', 'date', 'y', 'cp']
test_output.columns = ['code', 'region', 'date', 'y']
train_output['date'] = list(map(lambda x : datetime.strptime(x, '%Y-%m-%d %H'), train_output['date']))
test_output['date'] = list(map(lambda x : datetime.strptime(x, '%Y-%m-%d %H'), test_output['date']))

code = list(set(train_output['code']))
code.sort()

one_hot_encoder = OneHotEncoder()
code = np.array(code).reshape(-1,1)
one_hot_code = one_hot_encoder.fit_transform(code)
one_hot_code = one_hot_code.toarray()
code_to_one = {k:torch.tensor(v) for k, v in zip(one_hot_encoder.categories_[0], list(one_hot_code))}

train_output['one_hot_code'] = [code_to_one[x] for x in train_output['code']]
test_output['one_hot_code'] = [code_to_one[x] for x in test_output['code']]

total_table = pd.merge(tmp, gpm_table, on='date', how='inner')
total_table = total_table.sort_values(by='date')
total_table = pd.merge(total_table, train_output, on='date')

total_table_ = pd.merge(tmp, gpm_table, on='date', how='inner')
total_table_ = total_table_.sort_values(by='date')
test_table = pd.merge(total_table_, test_output, on='date')
test_table = test_table.sort_values(by='date')

total_table['year'] = total_table['date'].dt.year
total_table['month'] = total_table['date'].dt.month
test_table['year'] = test_table['date'].dt.year
test_table['month'] = test_table['date'].dt.month

In [ ]:
test_output = pd.read_csv('./test/sample_output_seomjingang.csv')
del test_output['비고']
test_output.columns = ['code', 'region', 'date', 'y']

In [ ]:
print('train & valid')
print('years :', set(total_table['year']))
print('months :', set(total_table['month']))
print()
print('test')
print('years :', set(test_table['year']))
print('months :', set(test_table['month']))

## valid_mask는 지역별로 1달씩 간격줘서 골고루 해도 괜찮을 것 같음. (나중에 구현)

In [ ]:
valid_mask = (total_table['year']==2020) & (total_table['month']==11) |\
    (total_table['year']==2021) & (total_table['month']==5)
train_mask = valid_mask==0

train_table = total_table[train_mask]
valid_table = total_table[valid_mask]

print('train :', train_table.shape)
print('valid :', valid_table.shape)
print('test :', test_table.shape)

In [ ]:
print('TPW 개수 :', len(tpw))
print('RR 개수 :', len(rr))
print('GPM 개수 :', len(gpm_h))
print('전체 개수 :', len(total_table))
print('결측치 개수 :', total_table.isna().sum().sum())
print('사용 데이터 개수 :', len(total_table))
print('학습용 데이터셋 :', len(train_table))
print('검증용 데이터셋 :', len(valid_table))
print('예측 데이터셋 :', len(test_table))
print()
print('테이블 열 목록')
print(total_table.columns)

# 섬진강 지역 Crop

In [ ]:
# geometric information
## Cheonlian 2A Image over all Korean Peninsula - 한반도 전역 천리안 이미지 지리정보
cheonlian_geo_files = glob.glob('./geometry/input-cheonlian-2A/*.dbf')
cheonlian_dbf = Dbf5(cheonlian_geo_files[0]).to_dataframe()
cheonlian_dbf['id'] = cheonlian_dbf['id'].map(lambda x: x-1)
## Seomjingang - 섬진강 유역의 관측소 지리정보
seomjingang_geo_files = glob.glob('./geometry/output-seomjingang/*.dbf')
seomjingang_dbf = Dbf5(seomjingang_geo_files[0]).to_dataframe()
# left upper, right lower coordinate - 좌상단 우하단 좌표
left_x, upper_y = seomjingang_dbf['x'].min(), seomjingang_dbf['y'].max()
right_x, lower_y = seomjingang_dbf['x'].max(), seomjingang_dbf['y'].min()
# get pixel indices of Seomjingang area from Cheonlian images
x_padding = 0.5
y_padding = 0.1
seomjingang_ids = cheonlian_dbf.loc[(cheonlian_dbf['x']>=left_x-x_padding) & (cheonlian_dbf['x']<=right_x+x_padding)
                      & (cheonlian_dbf['y']>=lower_y-y_padding) & (cheonlian_dbf['y']<=upper_y+y_padding)]['id']
x1 = min(seomjingang_ids)//900
x2 = max(seomjingang_ids)//900
y1 = min(map(lambda x: x%900, seomjingang_ids))
y2 = max(map(lambda x: x%900, seomjingang_ids))

# Dataset Class 생성

# 학습 방법 1. 한 모델로 전체 지점을 예측하는 모델을 만들자.
    - 지점 code를 one-hot vector로 만들어 입력

# ~학습 방법 2. 각 지점 별로 모델을 만들자.~

## 나중에 이미지 scaling 고려해야함

In [ ]:
class RainDataset(Dataset):
    def __init__(self, table, split='train'):
        super().__init__()
        self.table = table
        self.split = split
        self.tpw = self._get_tpw_image()
        self.rr = self._get_rr_image()
        self.gpm = self._get_gpm_image()
        self.code = list(table['one_hot_code'])
        if split != 'test':
            self.label = list(table['y'])

    def _get_tpw_image(self):
        tpws = []
        for path in self.table['tpw']:
            nc = netCDF4.Dataset(path, mode='r')
            image = nc.variables['TPW'][:]
            image = torch.tensor(image[x1:x2, y1:y2])
            tpws.append(image)
        return tpws
    
    def _get_rr_image(self):
        rrs = []
        for path in self.table['rr']:
            nc = netCDF4.Dataset(path, mode='r')
            image = nc.variables['RR'][:]
            image = torch.tensor(image[x1:x2, y1:y2])
            rrs.append(image)
        return rrs

    def _get_gpm_image(self):
        gpms = []
        for gpm_files in self.table['gpm']:
            gpm_mini = []
            gpm = torch.zeros((13, 11))
            for gpm_file in gpm_files: 
                with rasterio.open(gpm_file) as img :
                    gpm += np.reshape(img.read(), (13, 11))
                    gpm_mini.append(gpm)
            gpms.append(gpm_mini)
        return gpms

    def get_table(self):
        return self.table

    def __len__(self):
        return len(self.table)
    
    def __getitem__(self, idx):
        if self.split in ['train', 'valid']:
            return self.tpw[idx], self.rr[idx], self.gpm[idx], self.code[idx], self.label[idx]
        else:
            return self.tpw[idx], self.rr[idx], self.gpm[idx], self.code[idx]

In [ ]:
# train_dataset = RainDataset(train_table)
# valid_dataset = RainDataset(valid_table)
# torch.save(train_dataset, './train_dataset.pt')
# torch.save(valid_dataset, './valid_dataset.pt')
# test_dataset = RainDataset(test_table, split='test')

In [ ]:
train_dataset = torch.load('./train_dataset.pt') #train
valid_dataset = torch.load('./valid_dataset.pt')
test_dataset = torch.load('./test_dataset.pt')

In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

valid_loader = DataLoader(
    valid_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

# Backbone Network를 1:1:1로 할지 2:1로 할지 3으로 할지
## Resnet을 쓸지, 직접 짤지

In [ ]:
# net = timm.create_model('resnest50d_4s2x40d', pretrained=False, in_chans=2)
# net

In [ ]:
transforms_rr = transforms.Compose([
    transforms.Resize((224, 224)),
])

transforms_gpm = transforms.Compose([
    transforms.Resize((14, 14)),
])

In [ ]:
class Net(nn.Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.net = timm.create_model('resnest50d_4s2x40d', pretrained=False, in_chans=2)
        self.net.fc = nn.Sequential(
            nn.Linear(2048, 256),
            nn.ReLU()
        )
        self.gpm_cnn = nn.Sequential(
            # 2 * 14 * 14
            nn.Conv2d(2, 32, kernel_size=3, stride=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 1, kernel_size=3, stride=1),
        ) # 9
        
        self.linear_layers = nn.Sequential(
            nn.Linear(256+36+12, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(16, 1)
        )

    # Defining the forward pass    
    def forward(self, tpw_rr, gpm, one_hot_code):
        out1 = self.net(tpw_rr)
        out2 = self.gpm_cnn(gpm)
        out2 = out2.view(out2.size(0), -1)

        out = torch.cat([out1,out2,one_hot_code], axis=1)
        out = self.linear_layers(out)
        return out

In [ ]:
model = Net()
model = nn.DataParallel(model, output_device=1)
model.cuda()
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=1e-7, last_epoch=-1)
criterion = nn.MSELoss().to('cuda:3')

In [ ]:
train_error_e, valid_error_e = [], []
train_error, valid_error = [], []

best_models = []
best_score = 1.6

EPOCHS = 70

for e in range(EPOCHS):
    model.train()
    
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        tpw, rr, gpm, code, label = data
        tpw_rr = torch.stack([tpw,rr], axis=1)
        gpm = torch.stack(gpm, axis=1)
        tpw_rr = transforms_rr(tpw_rr)
        gpm = transforms_gpm(gpm)
        tpw_rr = tpw_rr.to(device)
        gpm = gpm.to(device)
        code = code.type(torch.float)
        code = code.to(device)
        label = label.type(torch.float)
        label = label.to('cuda:1')

        output = model(tpw_rr, gpm, code).squeeze()
        loss = torch.sqrt(criterion(output, label))
        loss.backward()
        optimizer.step()
        train_error.append(loss.item())
        running_loss += loss.item()*len(label)
    train_error_e.append(running_loss/len(train_dataset))

    running_loss = 0.0
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(valid_loader):
            tpw, rr, gpm, code, label = data
            tpw_rr = torch.stack([tpw,rr], axis=1)
            gpm = torch.stack(gpm, axis=1)
            tpw_rr = transforms_rr(tpw_rr)
            gpm = transforms_gpm(gpm)
            tpw_rr = tpw_rr.to(device)
            gpm = gpm.to(device)
            code = code.type(torch.float)
            code = code.to(device)
            label = label.type(torch.float)
            label = label.to('cuda:1')

            output = model(tpw_rr, gpm, code).squeeze()
            loss = torch.sqrt(criterion(output, label))
            valid_error.append(loss.item()/len(label))
            running_loss += loss.item()*len(label)
    valid_error_e.append(running_loss/len(valid_dataset))
    scheduler.step()
    print(f'[Epoch : {e}] Train : {round(train_error_e[-1], 3)} Valid : {round(valid_error_e[-1], 3)}')

    if valid_error_e[-1] < best_score:
        best_score = valid_error_e[-1]
        model_path = f'./model_{e}.pt'
        torch.save(model.module.state_dict(), model_path)
        best_models.append(model_path)
        print(valid_error_e[-1])

        model.eval()
        with torch.no_grad():
            for i, data in enumerate(test_loader):
                tpw, rr, gpm, code = data
                tpw_rr = torch.stack([tpw,rr], axis=1)
                gpm = torch.stack(gpm, axis=1)
                tpw_rr = transforms_rr(tpw_rr)
                gpm = transforms_gpm(gpm)
                tpw_rr = tpw_rr.to(device)
                gpm = gpm.to(device)
                code = code.type(torch.float)
                code = code.to(device)

                output = model(tpw_rr, gpm, code).squeeze()
                total.extend(output.to('cpu'))

        total = [x.item() for x in total]
        table = test_dataset.table
        table['y'] = total
        table['date'] = table['date'].astype(int)
        result_table = pd.read_csv('./test/sample_output_seomjingang.csv')
        result_table.columns =['code', 'region', 'date', 'y', 'etc']
        result_table['관측시간'] = result_table['date']
        result_table['date'] = list(map(lambda x : datetime.strptime(x, '%Y-%m-%d %H'), result_table['date']))
        result_table['date'] = result_table['date'].astype(int)
        output = pd.merge(result_table, table, on=['date','code'], how='left')
        output['y_y'] = output['y_y'].fillna(0)
        result_table = pd.read_csv('./test/sample_output_seomjingang.csv')
        result_table['시강수량'] = output['y_y']
        result_table.to_csv(f'./output_sample_{e}.csv', index=None)
        


plt.figure(figsize=(8,16))
plt.subplot(2,1,1)
plt.plot(train_error_e)
plt.plot(valid_error_e)
plt.show()


In [ ]:
m = './model_4.pt'
model=Net()
model.load_state_dict(torch.load(m))
model = nn.DataParallel(model, output_device=1)
model.cuda()
model.eval()

total = []
model.eval()
with torch.no_grad():
    for i, data in enumerate(test_loader):
        tpw, rr, gpm, code = data
        tpw_rr = torch.stack([tpw,rr], axis=1)
        gpm = torch.stack(gpm, axis=1)
        tpw_rr = transforms_rr(tpw_rr)
        gpm = transforms_gpm(gpm)
        tpw_rr = tpw_rr.to(device)
        gpm = gpm.to(device)
        code = code.type(torch.float)
        code = code.to(device)

        output = model(tpw_rr, gpm, code).squeeze()
        total.extend(output.to('cpu'))

total = [x.item() for x in total]
table = test_dataset.table
table['y'] = total
table['date'] = table['date'].astype(int)
result_table = pd.read_csv('./test/sample_output_seomjingang.csv')
result_table.columns =['code', 'region', 'date', 'y', 'etc']
result_table['관측시간'] = result_table['date']
result_table['date'] = list(map(lambda x : datetime.strptime(x, '%Y-%m-%d %H'), result_table['date']))
result_table['date'] = result_table['date'].astype(int)
output = pd.merge(result_table, table, on=['date','code'], how='left')
output['y_y'] = output['y_y'].fillna(0)
result_table = pd.read_csv('./test/sample_output_seomjingang.csv')
result_table['시강수량'] = output['y_y']
result_table.to_csv('./output_sample_e4.csv', index=None)

In [ ]:
total = [x.item() for x in total]

In [ ]:
table = test_dataset.table
table['y'] = total

In [ ]:
table['date']

In [ ]:
table['date'] = table['date'].astype(int)

In [ ]:
result_table = pd.read_csv('./test/sample_output_seomjingang.csv')
result_table.columns =['code', 'region', 'date', 'y', 'etc']
result_table['관측시간'] = result_table['date']
result_table['date'] = list(map(lambda x : datetime.strptime(x, '%Y-%m-%d %H'), result_table['date']))
result_table['date'] = result_table['date'].astype(int)
output = pd.merge(result_table, table, on=['date','code'], how='left')

In [ ]:
len(result_table)

In [ ]:
len(output)

In [ ]:
output.columns

In [ ]:
result_table = pd.read_csv('./test/sample_output_seomjingang.csv')
result_table['시강수량'] = output['y_y']

In [ ]:
result_table.to_csv('./output_sample1.csv', index=None)